In [1]:
import pickle
import networkx as nx
import numpy as np

In [2]:
with open('GOT-characters-raw.pickle','rb') as fp:
    characters = pickle.load(fp)
len(characters.keys())

340

In [3]:
entities = set()
for chapter in characters.values():
    for entity in chapter:
        name = entity[0]
        entities.add(name)
print('analysis contains',len(entities),'individual entities')

analysis contains 5522 individual entities


In [6]:
characters.keys()

dict_keys(['1–0-Prologue', '1–1-Bran', '1–2-Catelyn', '1–3-Daenerys', '1–4-Eddard', '1–5-Jon', '1–6-Catelyn', '1–7-Arya', '1–8-Bran', '1–9-Tyrion', '1–10-Jon', '1–11-Daenerys', '1–12-Eddard', '1–13-Tyrion', '1–14-Catelyn', '1–15-Sansa', '1–16-Eddard', '1–17-Bran', '1–18-Catelyn', '1–19-Jon', '1–20-Eddard', '1–21-Tyrion', '1–22-Arya', '1–23-Daenerys', '1–24-Bran', '1–25-Eddard', '1–26-Jon', '1–27-Eddard', '1–28-Catelyn', '1–29-Sansa', '1–30-Eddard', '1–31-Tyrion', '1–32-Arya', '1–33-Eddard', '1–34-Catelyn', '1–35-Eddard', '1–36-Daenerys', '1–37-Bran', '1–38-Tyrion', '1–39-Eddard', '1–40-Catelyn', '1–41-Jon', '1–42-Tyrion', '1–43-Eddard', '1–44-Sansa', '1–45-Eddard', '1–46-Daenerys', '1–47-Eddard', '1–48-Jon', '1–49-Eddard', '1–50-Arya', '1–51-Sansa', '1–52-Jon', '1–53-Bran', '1–54-Daenerys', '1–55-Catelyn', '1–56-Tyrion', '1–57-Sansa', '1–58-Eddard', '1–59-Catelyn', '1–60-Jon', '1–61-Daenerys', '1–62-Tyrion', '1–63-Catelyn', '1–64-Daenerys', '1–65-Arya', '1–66-Bran', '1–67-Sansa', '1–68

In [79]:
def get_edges_from_chapter(chapter):
    chapter.sort(key=lambda a:a[1],reverse=False)

    potential_edges = []
    for i in range(0,len(chapter)):
        name1,start1,end1 = chapter[i]
        for j in range(i+1,len(chapter)):    
            name2,start2,end2 = chapter[j]
            if name2 != name1:
                potential_edges.append((name1,name2,{'dist':start2-end1}))

    edges = {}
    for name1,name2,value in potential_edges:
        if (name1,name2) not in edges.keys():
            if (name2,name1) not in edges.keys():
                edges[(name1,name2)] = {'values':[value['dist']]}
            else:
                edges[(name2,name1)]['values'].append(value['dist'])
        else:
            edges[(name1,name2)]['values'].append(value['dist'])

    for k,v in edges.items():
        #edges[k]['weight']=np.percentile(v['values'],10)
        # the 448 value is the mean + standard deviation of the length of a paragraph in game of thrones books 
        edges[k]['weight']=len([a for a in v['values'] if a <448])
    
    ready_for_nx = []
    for k,v in edges.items():
        name1 = k[0]
        name2 = k[1]
        weight = v['weight']
        if weight > 0:
            ready_for_nx.append((name1,name2,{'weight':weight}))
    return ready_for_nx

In [82]:
GOT = nx.Graph()

In [83]:
for chapter in characters.values():
    GOT.add_edges_from(get_edges_from_chapter(chapter))

In [84]:
print(nx.info(GOT))

Name: 
Type: Graph
Number of nodes: 5515
Number of edges: 88598
Average degree:  32.1298


In [85]:
nx.write_gexf(GOT,'GOT-version-1.gexf')